# Bayesian computation in Python

## The case of simple linear regression

Herein follows a bit of a rant about Bayesian techniques (in preference over _unthinking_ application of hypothesis testing) and a quick demo of how to use `pymc3` and `emcee`.

Weapon of choice: Python 3 and Jupyter notebooks

# Linear regression

We start with a statistical model that we have an independent variable (or [input variable](https://en.wikipedia.org/wiki/Dependent_and_independent_variables) or predictor variable) $x$ which we know or control, and a dependence variable $y$ (or output variable or response variable) which we measure.  In simple linear regression we assume a linear, or straight line, relationship

$$ y = a + bx $$

As measurement might contain noise, or because we might believe this model to be a simplification of reality, and so there to be essentially random extra components, given data $(x_i)$ and $(y_i)$ we assume the relationship is

$$ y_i = a + bx_i + \epsilon_i $$

where $(\epsilon_i)$ are error terms.

We assume that the $(\epsilon_i)$ are _independent_ and _identically distributed_ with a normal distribution $N(0,\sigma^2)$.  That is, the mean of the error is $0$, and the standard deviation is $\sigma$.

# Input data

